In [1]:
import gym
import numpy as np
import tensorflow as tf
env = gym.make("FrozenLake-v0")
n_actions = env.action_space.n
input_dim = env.observation_space.n
model = tf.keras.Sequential() 
model.add(tf.keras.layers.Dense(64, input_dim = input_dim , activation = 'relu'))
model.add(tf.keras.layers.Dense(32, activation = 'relu'))
model.add(tf.keras.layers.Dense(n_actions, activation = 'linear'))
model.compile(optimizer=tf.keras.optimizers.Adam(), loss = 'mse')
def replay(replay_memory, minibatch_size=32):
    minibatch = np.random.choice(replay_memory, minibatch_size, replace=True)
    s_l =      np.array(list(map(lambda x: x['s'], minibatch)))
    a_l =      np.array(list(map(lambda x: x['a'], minibatch)))
    r_l =      np.array(list(map(lambda x: x['r'], minibatch)))
    sprime_l = np.array(list(map(lambda x: x['sprime'], minibatch)))
    done_l   = np.array(list(map(lambda x: x['done'], minibatch)))
    qvals_sprime_l = model.predict(sprime_l)
    target_f = model.predict(s_l) 
    for i,(s,a,r,qvals_sprime, done) in enumerate(zip(s_l,a_l,r_l,qvals_sprime_l, done_l)): 
        if not done:  target = r + gamma * np.max(qvals_sprime)
        else:         target = r
        target_f[i][a] = target
    model.fit(s_l,target_f, epochs=1, verbose=0)
    return model
n_episodes = 500
gamma = 0.99
epsilon = 0.9
minibatch_size = 32
r_sums = []  
replay_memory = []
mem_max_size = 100000

In [2]:
for n in range(n_episodes): 
    ss = env.reset()
    states_total = 16
    data = [[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]]
    def encode(data, states_total):
        targets = np.array(data).reshape(-1)
        return np.eye(states_total)[targets]
    m = encode(data,states_total)
    s = m[ss]
    #print(s)
    #print(len(s))
    done=False
    r_sum = 0
    while not done: 
        #env.render()
        qvals_s = model.predict(s.reshape(1,-1))
        if np.random.random() < epsilon:  a = env.action_space.sample()
        else:                             a = np.argmax(qvals_s); 
        sprime, r, done, info = env.step(a)
        r_sum += r
        q = encode(data,states_total)
        sprime = q[sprime]
        if len(replay_memory) > mem_max_size:
            replay_memory.pop(0)
        replay_memory.append({"s":s,"a":a,"r":r,"sprime":sprime,"done":done})
        #s = n[sprime]
        s=sprime
        model=replay(replay_memory, minibatch_size = minibatch_size)
    if epsilon > 0.001:      epsilon -= 0.001
    r_sums.append(r_sum)
    print(r_sum)
    print(epsilon)
    if n % 100 == 0: print(n)

0.0
0.899
0
0.0
0.898
0.0
0.897
0.0
0.896
0.0
0.895
0.0
0.894
0.0
0.893
0.0
0.892
0.0
0.891
0.0
0.89
0.0
0.889
0.0
0.888
0.0
0.887
0.0
0.886
0.0
0.885
0.0
0.884
0.0
0.883
0.0
0.882
0.0
0.881
0.0
0.88
0.0
0.879
0.0
0.878
0.0
0.877
0.0
0.876
0.0
0.875
0.0
0.874
0.0
0.873
0.0
0.872
0.0
0.871
0.0
0.87
0.0
0.869
0.0
0.868
0.0
0.867
0.0
0.866
0.0
0.865
0.0
0.864
0.0
0.863
0.0
0.862
0.0
0.861
0.0
0.86
0.0
0.859
0.0
0.858
0.0
0.857
0.0
0.856
0.0
0.855
0.0
0.854
0.0
0.853
0.0
0.852
0.0
0.851
0.0
0.85
0.0
0.849
0.0
0.848
0.0
0.847
0.0
0.846
1.0
0.845
0.0
0.844
0.0
0.843
0.0
0.842
0.0
0.841
0.0
0.84
0.0
0.839
0.0
0.838
0.0
0.837
0.0
0.836
0.0
0.835
0.0
0.834
0.0
0.833
0.0
0.832
0.0
0.831
0.0
0.83
0.0
0.829
0.0
0.828
0.0
0.827
0.0
0.826
0.0
0.825
1.0
0.824
0.0
0.823
0.0
0.822
0.0
0.821
0.0
0.82
0.0
0.819
0.0
0.818
0.0
0.817
0.0
0.816
0.0
0.815
0.0
0.814
0.0
0.813
0.0
0.8119999999999999
0.0
0.8109999999999999
0.0
0.8099999999999999
0.0
0.8089999999999999
0.0
0.8079999999999999
0.0
0.806999999999999